# Проверка гипотез
В примерах использованы синтетические данные.


Ниже приведены тесты на нормальность (распределение значений случайной величины в данной выборке - распределение Гаусса) и взаимосвязи между случайными величинами

In [ ]:
import numpy as np
data_uniform = np.random.rand(5000)
data_norm = np.random.randn(5000)
data_norm_ = np.random.randn(5000)*5 - 10

### Генерация тестовых данных

В этой ячейке создаются три массива синтетических данных:
- `data_uniform` - 5000 случайных чисел от 0 до 1 (равномерное распределение)
- `data_norm` - 5000 чисел из стандартного нормального распределения (среднее=0, std=1)
- `data_norm_` - нормальное распределение с масштабированием (*5) и сдвигом (-10)

Эти данные будут использоваться для демонстрации статистических тестов.

### Тест Shapiro-Wilk (основан на оценке дисперсии)

Для размера выборки N > 5000 значение статистики stat точно, но p-value может быть неточным.

Вероятность отклонить нуль-гипотезу около 5% в зависимости от размеров выборки https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html

### Тест Шапиро-Уилка на нормальность

Тест проверяет, соответствует ли распределение данных нормальному (гауссову) распределению.
- **Нулевая гипотеза H0:** данные имеют нормальное распределение
- **Альтернативная гипотеза H1:** данные не имеют нормального распределения

**Правило принятия решения:**
Если p-value < 0.05 → отвергаем H0 (данные не нормальные)
Если p-value ≥ 0.05 → нет оснований отвергнуть H0 (считаем данные нормальными)

In [ ]:
from scipy.stats import shapiro
data1 = data_norm
stat, p = shapiro(data1)
print(stat)
print(p)
if (p < 0.05):
    print("Гипотеза о нормальности распределения отклонена")
else:
    print("Нет оснований отклонить гипотезу о нормальности распределения")


0.9994746886823463
0.17715833395920244
Нет оснований отклонить гипотезу о нормальности распределения


Для нормальных данных p > 0.05 - нормальность не отвергается.

**Интерпретация результатов для нормальных данных:**
- p-value = 0.177 > 0.05
- Вывод: нет оснований отвергнуть гипотезу о нормальности
- Это ожидаемо, так как `data_norm` был сгенерирован из нормального распределения

In [ ]:
data2 = data_uniform
stat, p = shapiro(data2)
print(stat)
print(p)
if (p < 0.05):
    print("Гипотеза о нормальности распределения отклонена")
else:
    print("Нет оснований отклонить гипотезу о нормальности распределения")

0.9558229324346722
1.8624699438484243e-36
Гипотеза о нормальности распределения отклонена


Для нормальных данных p > 0.05 - нормальность не отвергается.

**Интерпретация результатов для равномерных данных:**
- p-value = 1.86e-36 (очень маленькое число) < 0.05
- Вывод: отвергаем гипотезу о нормальности
- Это ожидаемо, так как равномерное распределение не является нормальным

### Тест Андерсона-Дарлинга для разных законов распределения
 https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.anderson.html

Critical values provided are for the following significance levels:

normal/exponential
15%, 10%, 5%, 2.5%, 1%

**If the returned statistic is larger than these critical values then for the corresponding significance level, the null hypothesis that the data come from the chosen distribution can be rejected**.


In [ ]:
from scipy.stats import anderson
result = anderson(data1)
stat, critical_values, significance_level=result
print(stat)
print(critical_values)
print(significance_level)

0.5034931801310449
[0.576 0.655 0.786 0.917 1.091]
[15.  10.   5.   2.5  1. ]


In [ ]:
result = anderson(data2)
stat, critical_values, significance_level=result
print(stat)
print(critical_values)
print(significance_level)

54.279675016225156
[0.576 0.655 0.786 0.917 1.091]
[15.  10.   5.   2.5  1. ]


Тест Андерсона подтверждает выводы теста Шапиро.

### Тест Андерсона-Дарлинга

**Назначение:** Более мощный тест на нормальность, чем Шапиро-Уилка, особенно для больших выборок.

**Как интерпретировать:**
1. Сравниваем полученную статистику теста (`stat`) с критическими значениями (`critical_values`)
2. Критические значения соответствуют уровням значимости: 15%, 10%, 5%, 2.5%, 1%
3. **Правило:** Если статистика теста > критического значения → отвергаем гипотезу о нормальности на данном уровне значимости

**В нашем примере:**
Для `data1` (нормальные данные): статистика = 0.503, что меньше всех критических значений → данные нормальны на всех уровнях значимости.

Для `data2` (равномерные данные): статистика = 54.28, что значительно превышает все критические значения → данные не нормальны на всех уровнях значимости (даже на 1%).

### Коэффициент корреляции Пирсона

In [ ]:
from scipy.stats import pearsonr
data3 = data_norm_
corr, p = pearsonr(data1, data3)
print(corr)
print(p)

-0.007593650992072152
0.5913880199179341


In [ ]:
data4 =2*data2
corr, p = pearsonr(data4, data2)
print(corr)
print(p)

0.9999999999999998
0.0


### Коэффициент корреляции Пирсона

**Назначение:** Измеряет **линейную** связь между двумя непрерывными переменными.

**Диапазон значений:** от -1 до 1
- 1: полная прямая линейная связь
- -1: полная обратная линейная связь  
- 0: отсутствие линейной связи

**p-value:** Вероятность получить такую или более сильную корреляцию при условии, что на самом деле корреляции нет (H0: ρ = 0).

**Первый пример (data1 и data3):**
- Корреляция = -0.0076 (близко к 0) → практически нет линейной связи
- p-value = 0.591 > 0.05 → корреляция статистически не значима
- **Вывод:** Между data1 и data3 нет значимой линейной зависимости

**Второй пример (data4 и data2):**
- Корреляция = 1.0 → идеальная прямая линейная связь
- p-value = 0.0 < 0.05 → корреляция статистически значима
- **Вывод:** data4 и data2 полностью линейно связаны (что ожидаемо, так как data4 = 2 * data2)

Корреляция Пирсона показывает линейную связь. Первые данные не связаны, вторые - идеально связаны.

### Использование библиотеки pandas

In [ ]:
import pandas as pd
s1=pd.Series(data2)
s1

,0
0,0.896348
1,0.945445
2,0.197611
3,0.354619
4,0.632186
...,...
4995,0.403776
4996,0.109449
4997,0.733103
4998,0.073258


In [ ]:
s2=pd.Series(data4)
s2

,0
0,1.792696
1,1.890889
2,0.395221
3,0.709239
4,1.264372
...,...
4995,0.807551
4996,0.218897
4997,1.466206
4998,0.146515


In [ ]:
s1.corr(s2)

1.0

In [ ]:
s1.cov(s2)

0.16479053009530875

### Работа с библиотекой Pandas

Pandas предоставляет удобные методы для анализа данных:

1. **`.corr()`** - вычисляет коэффициент корреляции Пирсона между двумя Series
2. **`.cov()`** - вычисляет ковариацию между двумя Series

**Ковариация vs Корреляция:**
- Ковариация показывает направление связи (+ или -), но не её силу (зависит от масштаба данных)
- Корреляция нормирована от -1 до 1 и показывает силу связи независимо от масштаба

**В нашем примере:**
- `s1.corr(s2)` = 1.0 → полная линейная зависимость (ожидаемо, так как s2 = 2 * s1)
- `s1.cov(s2)` = 0.165 → положительная ковариация

### Коэффициент корреляции Спирмена (ранговая корреляция)

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html


In [ ]:
from scipy.stats import spearmanr
corr, p = spearmanr(range(200), [i*2 for i in range(200)])
print(corr)
print(p)

1.0
0.0


In [ ]:
list1 = np.array(range(200))
list1[list1%2]=np.random.randint(500,1000)
list2=[200 - i for i in range(200)]
corr, p = spearmanr(list1, list2)
print(corr)
print(p)

-0.9405981176773022
7.784272118124144e-95


### Коэффициент корреляции Спирмена

**Назначение:** Измеряет **монотонную** (не обязательно линейную) связь между переменными. Работает с рангами значений.

**Когда использовать вместо Пирсона:**
1. Когда данные не нормально распределены
2. Когда есть выбросы
3. Когда связь нелинейная, но монотонная (постоянно возрастает/убывает)

**Пример 1 (идеальная монотонная связь):**
- `range(200)` и `[i*2 for i in range(200)]` → линейная зависимость
- Корреляция Спирмена = 1.0 → идеальная монотонная связь

**Пример 2 (сильная, но не идеальная связь):**
- Между `list1` и `list2` корреляция = 0.941 → очень сильная обратная монотонная связь
- p-value = 7.78e-95 (очень близко к 0) → связь статистически значима

Корреляция Спирмена показывает монотонную связь. В примерах - сильные связи.

### Тест хи-квадрат
Проверяет, являются ли две категориальные переменные связанными или независимыми. Исходные данные - таблица частот для каждого значения.
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html

In [ ]:
from scipy.stats import chi2_contingency
table = np.array([[10, 10, 20], [20, 20, 20]])
stat, p, dof, expected = chi2_contingency(table)
print(stat)
print(p)

2.7777777777777777
0.24935220877729622


In [1]:
table = np.array([[100, 100, 100,100, 100], [150, 100, 100, 100, 200]])
stat, p, dof, expected = chi2_contingency(table)
print(stat)
print(p)

NameError: name 'np' is not defined

### Тест хи-квадрат для таблиц сопряженности

**Назначение:** Проверяет, есть ли статистически значимая связь между двумя категориальными переменными.

**Нулевая гипотеза H0:** Переменные независимы (нет связи)

**Альтернативная гипотеза H1:** Переменные зависимы (есть связь)

**Как работает:**
1. Сравнивает наблюдаемые частоты (исходные данные) с ожидаемыми частотами (если бы переменные были независимы)
2. Большие расхождения между наблюдаемыми и ожидаемыми частотами приводят к большой статистике χ²

**Первый пример:**
- p-value = 0.249 > 0.05 → нет оснований отвергнуть H0
- **Вывод:** Связь между переменными не обнаружена (они независимы)

**Второй пример:**
- p-value = 7.35e-05 < 0.05 → отвергаем H0
- **Вывод:** Существует статистически значимая связь между переменными

Хи-квадрат проверяет связь категориальных данных. В первом случае связи нет, во втором - есть.